导入相关的包
pip install pandas
pip install ddddocr
pip install selenium

In [17]:
import time
import ddddocr
from selenium.common import NoSuchElementException, InvalidSelectorException
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd


In [18]:
"""加载OCR，识别验证码用"""
ocr = ddddocr.DdddOcr(show_ad=False)

In [19]:
"""加载客户清单"""
# 13180
customers = pd.read_excel("tmp/customers.xlsx", dtype=str)
searchResult = pd.DataFrame(columns=['身份证', '法院名称', '地区名称', '立案时间', '案号',
                                     '做出执行依据单位', '生效法律文书确定的义务',
                                     '被执行人的履行情况', '失信被执行人行为具体情形'])

with open("tmp/pos", "r") as f:
    pos = str(f.read())
if len(pos) == 0:
    pos = 2
else:
    pos = int(pos)
print("从第 %d 行开始查询：" % pos)
pos -= 2


从第 12180 行开始查询：


In [20]:
"""selenium 启用缓存"""
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
# chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")


In [21]:
class AutoFetcher:
    def __init__(self, drv):
        self.driver = drv

    def openPage(self, url):
        self.driver.get(url)

    def inputCardNum(self, idcard):
        driver.find_element(By.NAME, "pCardNum").send_keys(idcard)

In [28]:
from selenium.common import TimeoutException

disruptCount = 0
logFile = open("tmp/log.log", "a")

with open("tmp/pos", "r") as f:
    pos = str(f.read())
if len(pos) == 0:
    pos = 2
else:
    pos = int(pos)
print("从第 %d 行开始查询：" % pos)
pos -= 2

driver = webdriver.Chrome(chrome_options=chrome_options)
auto = AutoFetcher(drv=driver)

for i in range(pos, customers.shape[0]):
    idcard = customers[customers.columns[0]][i]
    while True:
        try:
            driver.get("http://zxgk.court.gov.cn/shixin/")
            driver.find_element(By.NAME, "pCardNum").send_keys(idcard)
            break
        except NoSuchElementException:
            # print("%s is ok" % idcard)
            # logFile.write("%s is ok\n" % idcard)
            # logFile.flush()
            time.sleep(600)
        except TimeoutException:
            time.sleep(600)

    # driver.find_element(By.NAME, "pName").send_keys("陈华")

    while True:
        driver.find_element_by_id("captchaImg").click()
        time.sleep(2)
        img = driver.find_element_by_id("captchaImg")

        with open('tmp/1.png', 'wb') as file:
            file.write(img.screenshot_as_png)

        with open('tmp/1.png', 'rb') as f:
            img_bytes = f.read()
        check_code = ocr.classification(img_bytes)
        driver.find_element_by_id("yzm").clear()
        driver.find_element_by_id("yzm").send_keys(check_code)
        # driver.find_element_by_id("yzm").send_keys("ABCD")
        time.sleep(1)

        yzmWrong = driver.find_elements(By.XPATH, "//div/strong[text()='验证码错误!']/..")
        yzmRight = driver.find_element(By.XPATH, "//div/strong[text()='验证码正确!']/..")


        if not yzmRight.get_attribute("class").__contains__("hide") and yzmWrong[1].get_attribute("class").__contains__("hide"):
           break
        else:
            # print("验证码错误！")
            time.sleep(5)

    try:
        driver.find_element(By.XPATH, '//button[normalize-space()="查询"]').click()
        time.sleep(2)
        resultList = driver.find_elements(By.XPATH, '//a[text()="查看"]')
        if len(resultList) > 0:
            # for result in resultList[0:0]:
            result = resultList[0]
            result.click()
            time.sleep(1)
            courtNameDetail = driver.find_element_by_id("courtNameDetail").text
            areaNameDetail = driver.find_element_by_id("areaNameDetail").text
            regDateDetail = driver.find_element_by_id("regDateDetail").text
            caseCodeDetail = driver.find_element_by_id("caseCodeDetail").text
            gistUnitDetail = driver.find_element_by_id("gistUnitDetail").text
            dutyDetail = driver.find_element_by_id("dutyDetail").text
            performanceDetail = driver.find_element_by_id("performanceDetail").text
            disruptTypeNameDetail = driver.find_element_by_id("disruptTypeNameDetail").text

            print(idcard, courtNameDetail, areaNameDetail, areaNameDetail, regDateDetail, caseCodeDetail,
                  gistUnitDetail, performanceDetail, disruptTypeNameDetail)
            logFile.write("%s, %s, %s, %s, %s, %s, %s, %s, %s" % (idcard, courtNameDetail, areaNameDetail, areaNameDetail, regDateDetail, caseCodeDetail,
                  gistUnitDetail, performanceDetail, disruptTypeNameDetail))
            logFile.flush()

            # driver.find_element(By.XPATH, '//button[normalize-space()="关闭"]').click()
            logFile.write(idcard +"\n")
            logFile.flush()

            searchResult.loc[disruptCount] = [idcard, courtNameDetail, areaNameDetail, regDateDetail,
                                              caseCodeDetail, gistUnitDetail, dutyDetail, performanceDetail, disruptTypeNameDetail]
            disruptCount += 1
        # else:
            # print("%s is ok" % idcard)
    except InvalidSelectorException:
        print("%s is ok" % idcard)
    except NoSuchElementException:
        print("%s is ok" % idcard)

    with open("tmp/pos", "w") as f:
        f.write(str(i+2))
        f.flush()

    time.sleep(9)

with open("tmp/pos", "w") as f:
    f.write("2")
    f.flush()
searchResult.to_excel("tmp/查询结果.xlsx", index=None)
driver.close()

从第 2 行开始查询：


KeyboardInterrupt: 

SyntaxError: invalid syntax (<ipython-input-12-173d4ae8f8c2>, line 5)